# Big Assignment of Parallel Processing (Class A)
## Sales Transaction Data Analysis Using Pyspark
1. Michael Effendy (F1D022012)
2. Dara Finas Elen (F1D022038)
3. Nazila Imkani (F1D022084)
4. Wahyuni Sulastri (F1D022101)
5. Fadila Ramdhani Muaz (F1D022119)

Kaggle link for the dataset: https://www.kaggle.com/datasets/gabrielramos87/an-online-shop-business

Link for another google colab (without parallel processing) for time comparison: https://colab.research.google.com/drive/1anYJgJyB-b6BosFu6QP9GR9_TImEgLjO#scrollTo=tJSkWAhxqU68

In [3]:
import psutil
print(psutil.cpu_percent())

1.7


In [3]:
%pip install pyspark
%pip install mpi4py
print(psutil.cpu_percent())

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: C:\Users\yoga7.RAKASYA\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: C:\Users\yoga7.RAKASYA\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


NameError: name 'psutil' is not defined

In [5]:
from mpi4py import MPI
from pyspark.sql import SparkSession
import pandas as pd
import numpy as np
print(psutil.cpu_percent())

RuntimeError: cannot load MPI library
Could not find module 'C:\Users\yoga7.RAKASYA\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\DLLs' (or one of its dependencies). Try using the full path with constructor syntax.
Could not find module 'C:\Users\yoga7.RAKASYA\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Library\bin' (or one of its dependencies). Try using the full path with constructor syntax.
Could not find module 'C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\DLLs\impi.dll' (or one of its dependencies). Try using the full path with constructor syntax.
Could not find module 'C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\DLLs\msmpi.dll' (or one of its dependencies). Try using the full path with constructor syntax.
Could not find module 'C:\Program Files\WindowsApps\PythonSoftwa

In [4]:
spark = SparkSession.builder.appName("Game Sales Analysis").getOrCreate()
print(psutil.cpu_percent())

1.8


In [ ]:
import time
mulai_ukur = time.time()

In [5]:
df = spark.read.csv("game_info.csv", header=True, inferSchema=True)
print(psutil.cpu_percent())

2.0


In [7]:
df.show(10)
print(psutil.cpu_percent())

+------+--------------------+--------------------+----------+----------+-----+-------------------+--------------------+------+----------+--------+------------------+-------------+-----------------+-----------------+-------------+--------------------+--------------------+--------------------+--------------------+------------+----------------+------------------+-------------------+-------------------+--------------------+--------------------+
|    id|                slug|                name|metacritic|  released|  tba|            updated|             website|rating|rating_top|playtime|achievements_count|ratings_count|suggestions_count|game_series_count|reviews_count|           platforms|          developers|              genres|          publishers| esrb_rating|added_status_yet|added_status_owned|added_status_beaten|added_status_toplay|added_status_dropped|added_status_playing|
+------+--------------------+--------------------+----------+----------+-----+-------------------+------------

In [9]:
df.createOrReplaceTempView("sales")
print(psutil.cpu_percent())

0.7


In [ ]:
comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

# Load CSV hanya di rank 0
df = None
if rank == 0:
    df = pd.read_csv("games.csv")
    chunks = np.array_split(df, size)
else:
    chunks = None

# Broadcast potongan data
local_chunk = comm.scatter(chunks, root=0)

# FILTER: metacritic > 80
filtered_chunk = local_chunk[local_chunk["metacritic"] > 80]

# REDUCE: jumlah ratings_count
local_sum = filtered_chunk["ratings_count"].sum()
total_sum = comm.reduce(local_sum, op=MPI.SUM, root=0)

if rank == 0:
    print("Total ratings_count untuk metacritic > 80:", total_sum)
